In [1]:
import pandas as pd

burn_path = 'data/training/quarterly-6-by-6/satellite-burn/group_' + str(1) + '.csv'

burn = pd.read_csv(burn_path)
burn

,0.00
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [4]:
import numpy as np
y = np.genfromtxt(burn_path, delimiter=',')
y

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01,
       0.  , 0.  , 0.  ])

In [5]:
len(y)

36

In [9]:
weather_path = 'data/training/quarterly-6-by-6/weather/group_' + str(1) + '.csv'
weather = pd.read_csv(weather_path)
len(weather.values)

36

In [10]:
import pickle
file_name = "trained-models/2024-02-24_20-06-43_time-step-quarterly_fine-areas-36.pkl"
model = pickle.load(open(file_name, "rb"))
predictions = model.predict(weather.values)
predictions

array([  421.70355,   518.88367,  3002.3982 ,   781.6574 ,  1147.2286 ,
        1812.472  ,   563.3953 ,   617.2843 ,   -98.14897,   236.23538,
       10140.1    ,  1812.472  ,   705.4877 ,  1812.472  ,  1812.472  ,
        1812.472  , 12046.617  ,  1812.472  ,  1812.472  ,  1812.472  ,
        1812.472  ,  1812.472  ,  1024.2913 ,  1646.6821 ,    78.74858,
        1854.1311 ,   234.06042,   882.1093 ,  1226.4408 ,  4538.429  ,
         236.23538,  3266.5405 ,   234.06042,  1045.3296 ,   829.50214,
        4538.429  ], dtype=float32)

In [11]:
np.savetxt("foo.csv", predictions, delimiter="\n")